In [1]:
import numpy as np
import cv2
import time as t
import matplotlib.pyplot as plt
import math

In [2]:
def no(x):
    a = 0

In [3]:
def read_trackbars():
    hl = cv2.getTrackbarPos('h', 'low')
    sl = cv2.getTrackbarPos('s', 'low')
    vl = cv2.getTrackbarPos('v', 'low')
    
    hh = cv2.getTrackbarPos('h', 'high')
    sh = cv2.getTrackbarPos('s', 'high')
    vh = cv2.getTrackbarPos('v', 'high')

In [4]:
def setup_trackbars():
    cv2.namedWindow('low')
    cv2.namedWindow('high')
    cv2.createTrackbar('h', 'low', 0, 179, no)
    cv2.createTrackbar('s', 'low', 0, 255, no)
    cv2.createTrackbar('v', 'low', 0, 255, no)
    cv2.createTrackbar('h', 'high', 0, 179, no)
    cv2.createTrackbar('s', 'high', 0, 255, no)
    cv2.createTrackbar('v', 'high', 0, 255, no)
    cv2.createTrackbar('ratio_low', 'low', 100, 300, no)
    cv2.createTrackbar('ratio_high', 'high', 100, 300, no)
    cv2.createTrackbar('epsilon', 'low', 0, 100, no)
    cv2.createTrackbar('pp_low', 'low', 0, 1000, no)
    cv2.createTrackbar('pp_high', 'high', 0, 1000, no)

In [5]:
def draw_cross(img, x, y, length, color):
    img = cv2.line(img, (x, y), (x, y + length), color ,5)
    img = cv2.line(img, (x, y), (x, y - length), color ,5)
    img = cv2.line(img, (x, y), (x + length, y), color ,5)
    img = cv2.line(img, (x, y), (x - length, y), color ,5)
    return img

In [27]:
cap.release()
cv2.destroyAllWindows()

In [11]:
#cap.release()
#cv2.destroyAllWindows()
cap = cv2.VideoCapture(0)
#cap.set(3,320);
#cap.set(4,240);
#cap.set(3, 3280);
#cap.set(4, 2464);
#cap.set(3, 1280)
#cap.set(4, 720)
setup_trackbars()
hl = sl = vl = hh = sh = vh = 0
hight = [hh, sh, vh]
lowt = [hl, sl, vl]

lowi = np.zeros((200, 200, 3), np.uint8)
highi = np.zeros((200, 200, 3), np.uint8)
time2 = t.time()
t11 = t.time()
coun = 0

while True:
    __, img = cap.read()
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hl = cv2.getTrackbarPos('h', 'low')
    sl = cv2.getTrackbarPos('s', 'low')
    vl = cv2.getTrackbarPos('v', 'low')
    
    hh = cv2.getTrackbarPos('h', 'high')
    sh = cv2.getTrackbarPos('s', 'high')
    vh = cv2.getTrackbarPos('v', 'high')
    
    low_ratio = cv2.getTrackbarPos('ratio_low', 'low')/100
    high_ratio = cv2.getTrackbarPos('ratio_high', 'high')/100
    e = cv2.getTrackbarPos('epsilon', 'low')/100
    low_pp = cv2.getTrackbarPos('pp_low', 'low')
    high_pp = cv2.getTrackbarPos('pp_high', 'high')
    
    hight = np.array([hh, sh, vh], np.uint8)
    highi[:] = [hh, sh, vh]
    highc = cv2.cvtColor(highi, cv2.COLOR_HSV2BGR)
    
    lowt = np.array([hl, sl, vl], np.uint8)
    lowi[:] = [hl, sl, vl]
    lowc = cv2.cvtColor(lowi, cv2.COLOR_HSV2BGR)
    
    
    
    mask = cv2.inRange(hsv, lowt, hight)
    kernel = np.ones((5, 5), np.uint8)/25
    proc = cv2.erode(mask, kernel, 1)
    kernel = np.ones((10, 10), np.uint8)/100
    proc = cv2.dilate(proc, kernel, 1)
    #mask = proc
    proc = cv2.bitwise_and(img,img,mask = mask)
    
    img2, con, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(con) > 0:
        max_size = 0
        i_max_size = 0
        for cnt in con:
            area = cv2.contourArea(cnt)
            #print(area)
            if max_size < area:
                max_size = area
                i_max_size = 0
        #print(max_size, i_max_size)
        if max_size > 10:
            img = cv2.drawContours(img, con[i_max_size], -1, (0,255,0), 1)
            cnt = con[i_max_size]
            M = cv2.moments(cnt)
            cx = int(M['m10']/(M['m00']+0.001))
            cy = int(M['m01']/(M['m00']+0.001))
            img = draw_cross(img, cx, cy, 20, (0, 255, 0))
    
    coun += 1
    if t.time() - t11 >= 1:
        print(coun, t.time()-t11)
        coun = 0
        t11 = t.time()
    
    cv2.imshow('low', lowc)
    cv2.imshow('high', highc)
    cv2.imshow('img', img)
    cv2.imshow('mask', mask)
    cv2.imshow('proc', proc)
    
    if cv2.waitKey(1) != -1:
        break

cap.release()
cv2.destroyAllWindows()

2 1.0881848335266113
18 1.034886360168457
16 1.009242057800293
15 1.0304126739501953
18 1.0409801006317139
17 1.0231237411499023
19 1.0521824359893799
17 1.037529706954956
18 1.0013329982757568
13 1.0579631328582764
12 1.0099310874938965
13 1.0272984504699707
13 1.0095798969268799
13 1.0421507358551025
13 1.0456326007843018
13 1.0435326099395752
13 1.05696702003479
12 1.010340690612793
16 1.004033088684082
16 1.0601990222930908
16 1.0061664581298828
16 1.0532703399658203
18 1.037099838256836
17 1.0007901191711426
16 1.01475191116333
15 1.015064001083374
12 1.0200397968292236
13 1.0660717487335205
13 1.0772764682769775
13 1.0882751941680908
13 1.0049958229064941
13 1.050924301147461
14 1.0164356231689453
14 1.0464119911193848
15 1.039231300354004
15 1.0361683368682861
14 1.04388427734375
14 1.0149526596069336
15 1.0665507316589355
15 1.0208549499511719
15 1.032285213470459
15 1.003854751586914
15 1.017838478088379
15 1.0288522243499756
16 1.055443286895752
15 1.024451494216919
15 1.0215

In [6]:
def n_turns(cont):
    c = len(cont)
    angl = 0
    n = 0
    for i in range (c):
        k1, l1 = line(cont[i], cont[(i+1)%c])
        k2, l2 = line(cont[(i+1)%c], cont[(i+2)%c])
        k3, l3 = line(cont[i], cont[(i+2)%c])
        anglp, temp = find_angle(k1, k2, k3, l3, cont[i], cont[(i+1)%c], cont[(i+2)%c])
        n+= temp
        angl += anglp
    #print("n=", n)
    return angl
        

In [7]:
def line(a, b):
    a = a[0]
    b = b[0]
    #print("a=", a, " b=", b)
    if (b[0]-a[0]) != 0:
        k = (b[1]-a[1])/(b[0]-a[0])
        l = a[1] - k*a[0]
        if k > 100000:
            return k, a[0]
        return k, l
    else:
        return 100000, a[0] 

In [8]:
def find_angle(k1, k2, k3, l3, a, b, c):
    a = a[0]
    b = b[0]
    c = c[0]
    anglr = 0
    n = 0
    if (1+k1*k2) != 0:
        anglr = math.atan(abs((k1-k2)/(1+k1*k2)))
    else:
        anglr = math.pi / 2
    
    if a[0] < c[0]:
        if k1 < 100000:
            if b[1] < k3*b[0]+l3:
                anglr = 2*math.pi - anglr
                n+= 1
        else:
            if b[0] > l3:
                anglr = 2*math.pi - anglr
                n+= 1
                
    else:
        if k1 < 100000:
            if b[1] > k3*b[0]+l3:
                anglr = 2*math.pi - anglr
                n+= 1
        else:
            if b[0] < l3:
                anglr = 2*math.pi - anglr
                n+= 1
    print("anglr=", math.degrees(anglr))#, " y=", k1, k2, k3, l3, a, b, c)
    return anglr, n
    

In [9]:
def list_crt(c, n):
    ret = []
    for i in range (n):
        ret.append(c[i])
    return ret